In [138]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['dwell', 'flow_in', 'flow_out']
for label in labels:
    train = []
    test = []
    for va in flow[['city_code','district_code']].drop_duplicates().values:
        cc = va[0]
        dc = va[1]
        price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
        k = 274
        diff = 15
        l = [cc,dc]
        l.extend(price[274-diff:274])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [cc,dc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=False)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='dwell':
        sub = test_x[[0,1]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['city_code'] = sub[0].map(cc_se)
sub['district_code'] = sub[1].map(dc_se)

sub = sub[flow.columns[1:]]

df = pd.DataFrame()
for i in range(15):
    temp = sub.copy()
    temp['date_dt'] = 20180302+i
    df = pd.concat([df,temp],axis=0,ignore_index=True)
df = df[flow.columns]
print('\n')
print(rs)
print(np.array(rs).mean())
df.to_csv('result/submission_1354.csv',index=False,header=None)
线上成绩 0.1354

(24010, 17)
(98, 17)
[37.548730025778696, 40.95914731155888, 36.18794590647185, 41.194913946184236, 38.41190724258504]
38.860528886515745
(24010, 17)
(98, 17)
[62.758456836186966, 67.08315708856169, 61.33884191891694, 67.81396020088431, 62.53101885188041]
64.30508697928606
(24010, 17)
(98, 17)
[64.4891045046544, 66.79329805458046, 60.71131468056843, 71.81777604983758, 67.6564313296846]
66.2935849238651


[38.860528886515745, 64.30508697928606, 66.2935849238651]
56.4864002632223


In [103]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['dwell', 'flow_in', 'flow_out']
for label in labels:
    train = []
    test = []
    for va in flow[['city_code','district_code']].drop_duplicates().values:
        cc = va[0]
        dc = va[1]
        price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
        k = 274
        diff = 15
        l = [cc,dc]
        l.extend(price[274-diff:274])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [cc,dc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])
    
    def getFeature(dt):
        cols = dt.columns[2:]
        dt['mean'] = dt[cols].mean(1)
        dt['std'] = dt[cols].std(1)
        dt['max'] = dt[cols].max(1)
        dt['min'] = dt[cols].min(1)
        return dt
    train_x = getFeature(train_x)
    test_x = getFeature(test_x)

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=False)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='dwell':
        sub = test_x[[0,1]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['city_code'] = sub[0].map(cc_se)
sub['district_code'] = sub[1].map(dc_se)

sub = sub[flow.columns[1:]]

df = pd.DataFrame()
for i in range(15):
    temp = sub.copy()
    temp['date_dt'] = 20180302+i
    df = pd.concat([df,temp],axis=0,ignore_index=True)
df = df[flow.columns]
print('\n')
print(rs)
print(np.array(rs).mean())
df.to_csv('result/submission_1509.csv',index=False,header=None)
线上成绩 0.1509

(24010, 21)
(98, 21)
[29.997067131683, 33.08260324441701, 27.0393425754335, 31.503929249926525, 29.648572636283]
30.254302967548607
(24010, 21)
(98, 21)
[46.712738522226836, 43.69355908681719, 42.084943583360705, 44.0931880452001, 45.914475345089386]
44.499780916538846
(24010, 21)
(98, 21)
[47.09570408003249, 43.80955154500376, 40.0933302867709, 50.83205006626566, 46.46731177498548]
45.659589550611656


[30.254302967548607, 44.499780916538846, 45.659589550611656]
40.137891144899704


In [104]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['dwell', 'flow_in', 'flow_out']
for label in labels:
    train = []
    test = []
    for va in flow[['city_code','district_code']].drop_duplicates().values:
        cc = va[0]
        dc = va[1]
        price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
        k = 274
        diff = 15
        l = [cc,dc]
        l.extend(price[274-diff:274])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [cc,dc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    train_weight = len(train_x)-pd.Series(train_x.index)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=False)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(1)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='dwell':
        sub = test_x[[0,1]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['city_code'] = sub[0].map(cc_se)
sub['district_code'] = sub[1].map(dc_se)

sub = sub[flow.columns[1:]]

df = pd.DataFrame()
for i in range(15):
    temp = sub.copy()
    temp['date_dt'] = 20180302+i
    df = pd.concat([df,temp],axis=0,ignore_index=True)
df = df[flow.columns]
print('\n')
print(rs)
print(np.array(rs).mean())
df.to_csv('result/submission_1380.csv',index=False,header=None)

线上得分 0.1380  (按照新旧来调整权重)

(24010, 17)
(98, 17)
[30.523645479760486, 32.648932167506906, 32.529010555686504, 29.532793930338602, 29.50927055677492]
30.948730538013486
(24010, 17)
(98, 17)
[49.260212644029, 51.317885040515876, 49.73833383034031, 47.29280421428015, 49.29925055268988]
49.38169725637104
(24010, 17)
(98, 17)
[50.10887032579559, 51.09190942048115, 51.76389892280361, 51.15056823618338, 51.39565953644155]
51.10218128834105


[30.948730538013486, 49.38169725637104, 51.10218128834105]
43.81086969424186


In [151]:
##细分进行整理预测，最后再整合

import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')
print(flow.columns)
print(trans.columns)

cc_se = pd.Series(trans['o_city_code'].value_counts().index)
dc_se = pd.Series(trans['o_district_code'].value_counts().index)

trans['o_city_code'] = trans['o_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['o_district_code'] = trans['o_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

trans['d_city_code'] = trans['d_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['d_district_code'] = trans['d_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['cnt']
for label in labels:
    train = []
    test = []
    for va in trans[['o_city_code','o_district_code','d_city_code','d_district_code']].drop_duplicates().values:
        occ = va[0]
        odc = va[1]
        dcc = va[2]
        ddc = va[3]
        price = trans[(trans['o_city_code']==occ)&(trans['o_district_code']==odc)&(trans['d_city_code']==dcc)&(trans['d_district_code']==ddc)][label].values.tolist()
        k = len(price)
        diff = 15
        l = [occ,odc,dcc,ddc]
        l.extend(price[k-diff:k])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [occ,odc,dcc,ddc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=VBE)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='cnt':
        sub = test_x[[0,1,2,3]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['o_city_code'] = sub[0].map(cc_se)
sub['o_district_code'] = sub[1].map(dc_se)
sub['d_city_code'] = sub[2].map(cc_se)
sub['d_district_code'] = sub[3].map(dc_se)

res = pd.read_csv('result/ronghe.csv',header=None)
res.columns = flow.columns.tolist()
res['flow_out'] = res['district_code'].map(sub.groupby('o_district_code')['cnt'].sum())
res['flow_in'] = res['district_code'].map(sub.groupby('d_district_code')['cnt'].sum())
res.to_csv('result/submission_part_1300.csv',header=None,index=False)
线上得分 1300

Index(['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in',
       'flow_out'],
      dtype='object')
Index(['date_dt', 'o_city_code', 'o_district_code', 'd_city_code',
       'd_district_code', 'cnt'],
      dtype='object')
(2204660, 19)
(9506, 19)
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[553]	train's rmse: 2.71742	evals's rmse: 3.54065
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[346]	train's rmse: 2.89298	evals's rmse: 3.44232
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[587]	train's rmse: 2.77647	evals's rmse: 3.30143
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[556]	train's rmse: 2.72438	evals's rmse: 3.51541
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[476]	train's rmse: 2.79212	evals's rmse: 3.55074
[3

In [152]:
##细分进行整理预测，最后再整合

import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')
print(flow.columns)
print(trans.columns)

cc_se = pd.Series(trans['o_city_code'].value_counts().index)
dc_se = pd.Series(trans['o_district_code'].value_counts().index)

trans['o_city_code'] = trans['o_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['o_district_code'] = trans['o_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

trans['d_city_code'] = trans['d_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['d_district_code'] = trans['d_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['cnt']
for label in labels:
    train = []
    test = []
    for va in trans[['o_city_code','o_district_code','d_city_code','d_district_code']].drop_duplicates().values:
        occ = va[0]
        odc = va[1]
        dcc = va[2]
        ddc = va[3]
        price = trans[(trans['o_city_code']==occ)&(trans['o_district_code']==odc)&(trans['d_city_code']==dcc)&(trans['d_district_code']==ddc)][label].values.tolist()
        k = len(price)
        diff = 30
        l = [occ,odc,dcc,ddc]
        l.extend(price[k-diff:k])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [occ,odc,dcc,ddc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=VBE)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='cnt':
        sub = test_x[[0,1,2,3]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['o_city_code'] = sub[0].map(cc_se)
sub['o_district_code'] = sub[1].map(dc_se)
sub['d_city_code'] = sub[2].map(cc_se)
sub['d_district_code'] = sub[3].map(dc_se)

res = pd.read_csv('result/ronghe.csv',header=None)
res.columns = flow.columns.tolist()
res['flow_out'] = res['district_code'].map(sub.groupby('o_district_code')['cnt'].sum())
res['flow_in'] = res['district_code'].map(sub.groupby('d_district_code')['cnt'].sum())
res.to_csv('result/submission_part_30.csv',header=None,index=False)
1307

Index(['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in',
       'flow_out'],
      dtype='object')
Index(['date_dt', 'o_city_code', 'o_district_code', 'd_city_code',
       'd_district_code', 'cnt'],
      dtype='object')
(2062164, 34)
(9506, 34)
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[325]	train's rmse: 2.90902	evals's rmse: 3.45365
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[445]	train's rmse: 2.49575	evals's rmse: 3.23916
Training until validation scores don't improve for 200 rounds.
[1000]	train's rmse: 2.25778	evals's rmse: 3.33884
Early stopping, best iteration is:
[823]	train's rmse: 2.31302	evals's rmse: 3.33802
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[786]	train's rmse: 2.50514	evals's rmse: 3.29057
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[

In [154]:
##细分进行整理预测，最后再整合

import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')
print(flow.columns)
print(trans.columns)

cc_se = pd.Series(trans['o_city_code'].value_counts().index)
dc_se = pd.Series(trans['o_district_code'].value_counts().index)

trans['o_city_code'] = trans['o_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['o_district_code'] = trans['o_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

trans['d_city_code'] = trans['d_city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
trans['d_district_code'] = trans['d_district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['cnt']
for label in labels:
    train = []
    test = []
    for va in trans[['o_city_code','o_district_code','d_city_code','d_district_code']].drop_duplicates().values:
        occ = va[0]
        odc = va[1]
        dcc = va[2]
        ddc = va[3]
        price = trans[(trans['o_city_code']==occ)&(trans['o_district_code']==odc)&(trans['d_city_code']==dcc)&(trans['d_district_code']==ddc)][label].values.tolist()
        k = len(price)
        diff = 45
        l = [occ,odc,dcc,ddc]
        l.extend(price[k-diff:k])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [occ,odc,dcc,ddc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=VBE)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='cnt':
        sub = test_x[[0,1,2,3]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['o_city_code'] = sub[0].map(cc_se)
sub['o_district_code'] = sub[1].map(dc_se)
sub['d_city_code'] = sub[2].map(cc_se)
sub['d_district_code'] = sub[3].map(dc_se)

res = pd.read_csv('result/ronghe.csv',header=None)
res.columns = flow.columns.tolist()
res['flow_out'] = res['district_code'].map(sub.groupby('o_district_code')['cnt'].sum())
res['flow_in'] = res['district_code'].map(sub.groupby('d_district_code')['cnt'].sum())
res.to_csv('result/submission_part_45.csv',header=None,index=False)

Index(['date_dt', 'city_code', 'district_code', 'dwell', 'flow_in',
       'flow_out'],
      dtype='object')
Index(['date_dt', 'o_city_code', 'o_district_code', 'd_city_code',
       'd_district_code', 'cnt'],
      dtype='object')
(1919808, 49)
(9506, 49)
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[679]	train's rmse: 2.13908	evals's rmse: 3.00121
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[535]	train's rmse: 2.10133	evals's rmse: 3.02902
Training until validation scores don't improve for 200 rounds.
[1000]	train's rmse: 1.8176	evals's rmse: 2.86349
Early stopping, best iteration is:
[886]	train's rmse: 1.86009	evals's rmse: 2.86057
Training until validation scores don't improve for 200 rounds.
Early stopping, best iteration is:
[472]	train's rmse: 2.23311	evals's rmse: 3.17907
Training until validation scores don't improve for 200 rounds.
[1000]	train's rmse: 1.76091	evals's 

In [155]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['dwell', 'flow_in', 'flow_out']
for label in labels:
    train = []
    test = []
    for va in flow[['city_code','district_code']].drop_duplicates().values:
        cc = va[0]
        dc = va[1]
        price = (np.log(flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label])+1).values.tolist()
        k = 274
        diff = 15
        l = [cc,dc]
        l.extend(price[274-diff:274])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [cc,dc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=False)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='dwell':
        sub = test_x[[0,1]]
        sub[label] = np.exp(df.mean(1))-1
    else:
        sub[label] = np.exp(df.mean(1))-1

sub['city_code'] = sub[0].map(cc_se)
sub['district_code'] = sub[1].map(dc_se)

sub = sub[flow.columns[1:]]

df = pd.DataFrame()
for i in range(15):
    temp = sub.copy()
    temp['date_dt'] = 20180302+i
    df = pd.concat([df,temp],axis=0,ignore_index=True)
df = df[flow.columns]
print('\n')
print(rs)
print(np.array(rs).mean())
df.to_csv('result/submission_log.csv',index=False,header=None)
0.9411

(24010, 17)
(98, 17)
[0.09405509453763258, 0.09474725415169521, 0.09622023398330724, 0.09229262310191763, 0.09992271199523652]
0.09544758355395784
(24010, 17)
(98, 17)
[0.12853361638556618, 0.12918514799718472, 0.12886322026256308, 0.12491771122288328, 0.13260764134143851]
0.12882146744192716
(24010, 17)
(98, 17)
[0.1266660458213939, 0.12894302723296383, 0.12618940832668246, 0.12280397508226473, 0.13080389871322823]
0.12708127103530664


[0.09544758355395784, 0.12882146744192716, 0.12708127103530664]
0.11711677401039722


In [156]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['dwell', 'flow_in', 'flow_out']
for label in labels:
    train = []
    test = []
    for va in flow[['city_code','district_code']].drop_duplicates().values:
        cc = va[0]
        dc = va[1]
        price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
        k = 274
        diff = 30
        l = [cc,dc]
        l.extend(price[274-diff:274])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [cc,dc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=False)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='dwell':
        sub = test_x[[0,1]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['city_code'] = sub[0].map(cc_se)
sub['district_code'] = sub[1].map(dc_se)

sub = sub[flow.columns[1:]]

df = pd.DataFrame()
for i in range(15):
    temp = sub.copy()
    temp['date_dt'] = 20180302+i
    df = pd.concat([df,temp],axis=0,ignore_index=True)
df = df[flow.columns]
print('\n')
print(rs)
print(np.array(rs).mean())
df.to_csv('result/submission_30.csv',index=False,header=None)

0.1329
根据特征的窗口长短发现得分是不一样的
只利用flow数据

(22540, 32)
(98, 32)
[36.548251476399656, 38.869706848823064, 40.119628545750146, 43.14896751102394, 42.32019697378858]
40.20135027115708
(22540, 32)
(98, 32)
[55.13252088746132, 60.76695047824509, 58.179226759553224, 65.78219464388503, 67.35388877122035]
61.44295630807299
(22540, 32)
(98, 32)
[60.41599287735531, 62.209846516490074, 62.13098252080111, 77.73605501932836, 71.42685620085761]
66.7839466269665


[40.20135027115708, 61.44295630807299, 66.7839466269665]
56.142751068732196


In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))
rmse_dff = []
for dff in range(5,100):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = 274
            diff = dff
            l = [cc,dc]
            l.extend(price[274-diff:274])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-15-diff:k-15])
                l.append(sum(price[k-15:k])/15)
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':1, 
            'bagging_freq':1,
            'feature_fraction':1, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            print(train_x.shape)
            print(test_x.shape)
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        print(rmse)
        print(np.array(rmse).mean())
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    sub['city_code'] = sub[0].map(cc_se)
    sub['district_code'] = sub[1].map(dc_se)

    sub = sub[flow.columns[1:]]

    df = pd.DataFrame()
    for i in range(15):
        temp = sub.copy()
        temp['date_dt'] = 20180302+i
        df = pd.concat([df,temp],axis=0,ignore_index=True)
    df = df[flow.columns]
    print('\n')
    print(rs)
    print(np.array(rs).mean())
    rmse_dff.append(np.array(rs).mean())
pd.Series(rmse_dff,index=range(5,100)).sort_values().head()

(24990, 7)
(98, 7)
[44.60320302753828, 43.06192449608929, 37.582302914631136, 45.446768753613654, 40.404940386254594]
42.219827915625395
(24990, 7)
(98, 7)
[75.70384996461586, 71.0905244698963, 63.49940823578407, 75.24297750592805, 69.05621078454134]
70.91859419215312
(24990, 7)
(98, 7)
[74.71438519790355, 73.25884159705485, 63.48277018790797, 71.78626472942199, 65.77344095918768]
69.80314053429521


[42.219827915625395, 70.91859419215312, 69.80314053429521]
60.980520880691245
(24892, 8)
(98, 8)
[40.07020616880584, 42.41742157053947, 35.364464724734965, 43.41884078322204, 43.155599819115345]
40.88530661328353
(24892, 8)
(98, 8)
[66.609284200454, 77.76714901506367, 62.149277985298305, 71.69402048448471, 73.19991826268634]
70.28392998959741
(24892, 8)
(98, 8)
[65.60688318772142, 74.7379389387474, 62.26905406096199, 70.77911211988713, 71.23750785455198]
68.92609923237399


[40.88530661328353, 70.28392998959741, 68.92609923237399]
60.03177861175164
(24794, 9)
(98, 9)
[43.7853185037692, 39.

(23324, 24)
(98, 24)
[42.30440043916645, 38.40191326390623, 36.92019173910041, 40.81846000933282, 38.11340172476352]
39.31167343525389
(23324, 24)
(98, 24)
[66.07286417853277, 70.4862073564029, 61.39789623914076, 64.62413534036787, 63.62364254824053]
65.24094913253697
(23324, 24)
(98, 24)
[68.3543192963832, 71.19538641455219, 60.79665321712838, 69.78325922060546, 63.98616611428719]
66.82315685259128


[39.31167343525389, 65.24094913253697, 66.82315685259128]
57.12525980679404
(23226, 25)
(98, 25)
[42.92015925492264, 39.26271274431123, 36.496936349364525, 40.37235440153327, 38.195007917324276]
39.44943413349118
(23226, 25)
(98, 25)
[66.57404457787221, 67.44872120165026, 60.51994911226434, 68.84145809954458, 60.90646090890752]
64.8581267800478
(23226, 25)
(98, 25)
[72.18858266501988, 69.25959157092024, 60.27191328870604, 76.13905339448287, 63.69391084493967]
68.31061035281374


[39.44943413349118, 64.8581267800478, 68.31061035281374]
57.53939042211757
(23128, 26)
(98, 26)
[41.14212845198

(21658, 41)
(98, 41)
[37.080064242832876, 32.99448766612478, 41.3527800884629, 36.37387931993493, 36.32296167185397]
36.82483459784189
(21658, 41)
(98, 41)
[59.110137834991924, 50.565497856742894, 63.96162905027579, 59.238695674847044, 59.0789643144813]
58.39098494626778
(21658, 41)
(98, 41)
[62.71172003601982, 54.01397053918176, 68.0051372543446, 59.974798615114025, 63.21572984522693]
61.58427125797742


[36.82483459784189, 58.39098494626778, 61.58427125797742]
52.26669693402903
(21560, 42)
(98, 42)
[37.6879128039351, 32.72015146780306, 38.741526212014854, 39.352310908012115, 33.56772630485211]
36.41392553932345
(21560, 42)
(98, 42)
[55.3234559013161, 53.947116674263256, 61.646133028627375, 59.61863478475168, 54.32483369349856]
56.972034816491394
(21560, 42)
(98, 42)
[62.19710521843379, 53.895166495211974, 62.195704465632396, 60.61377622874611, 58.285668931436334]
59.43748426789212


[36.41392553932345, 56.972034816491394, 59.43748426789212]
50.94114820790232
(21462, 43)
(98, 43)
[37.

KeyboardInterrupt: 

In [4]:
pd.Series(rmse_dff,index=range(5,45)).sort_values().head()

44    49.566224
43    50.032059
42    50.567430
40    50.941148
41    51.124800
dtype: float64

In [5]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

rs = []
labels =['dwell', 'flow_in', 'flow_out']
for label in labels:
    train = []
    test = []
    for va in flow[['city_code','district_code']].drop_duplicates().values:
        cc = va[0]
        dc = va[1]
        price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
        k = 274
        diff = 45
        l = [cc,dc]
        l.extend(price[274-diff:274])
        l.append(np.nan)
        test.append(l)
        while k-15-diff>=0:
            l = [cc,dc]
            l.extend(price[k-15-diff:k-15])
            l.append(sum(price[k-15:k])/15)
            k = k-1
            train.append(l)

    test_x = pd.DataFrame(test)
    train_x = pd.DataFrame(train)
    del test_x[train_x.columns[-1]]
    train_y = train_x.pop(train_x.columns[-1])

    params_default_lgb = {
            'num_leaves':80, 
            'learning_rate':0.1, 
        'boosting':'gbdt',
        'min_child_samples':20,

        'bagging_fraction':1, 
        'bagging_freq':1,
        'feature_fraction':1, 
         'max_depth':-1,
        'reg_alpha':2,
        'reg_lambda':5, 
        'metric':'rmse',
        'objective':'regression'
    }
    NBR = 20000
    VBE = 1000
    ESR = 200
    def get_predict_result(k):
        print(train_x.shape)
        print(test_x.shape)
        df = pd.DataFrame()
        num = 0
        rmse = []
        skf = KFold(n_splits=5,random_state=2020,shuffle=True)
        for train_part_index,evals_index in skf.split(train_x,train_y):
            if k==1:
                train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
            elif k==0:
                train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
            bst = lgb.train(params_default_lgb,train_part, 
                  num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                  valid_names=['train','evals'],early_stopping_rounds=ESR,
                     verbose_eval=False)
            num+=1
            valid_ypre = bst.predict(test_x)
            df['pred_'+str(num)]=valid_ypre
            rmse.append(bst.best_score['evals']['rmse'])
        return df,rmse
    df,rmse = get_predict_result(0)
    print(rmse)
    print(np.array(rmse).mean())
    rs.append(np.array(rmse).mean())
    if label=='dwell':
        sub = test_x[[0,1]]
        sub[label] = df.mean(1)
    else:
        sub[label] = df.mean(1)

sub['city_code'] = sub[0].map(cc_se)
sub['district_code'] = sub[1].map(dc_se)

sub = sub[flow.columns[1:]]

df = pd.DataFrame()
for i in range(15):
    temp = sub.copy()
    temp['date_dt'] = 20180302+i
    df = pd.concat([df,temp],axis=0,ignore_index=True)
df = df[flow.columns]
print('\n')
print(rs)
print(np.array(rs).mean())
df.to_csv('result/submission_'+str(diff)+'.csv',index=False,header=None)
0.1376

(21070, 47)
(98, 47)
[32.593453600143334, 33.78443267559931, 29.697849844154277, 35.203028684412544, 37.49560088537886]
33.75487313793766
(21070, 47)
(98, 47)
[53.343317147190085, 52.42950374732676, 51.49024387609892, 54.64942731218163, 57.26387526096998]
53.835273468753485
(21070, 47)
(98, 47)
[57.729149196627176, 54.223746192632156, 54.599906896200274, 60.294030604239246, 60.76179165623929]
57.521724909187625


[33.75487313793766, 53.835273468753485, 57.521724909187625]
48.37062383862625


In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

flow = pd.read_csv('data/flow_train.csv')

valid = flow['date_dt'].drop_duplicates().sort_values().values[-15]
valid = flow[flow['date_dt']>=valid].reset_index(drop=True)
trans = pd.read_csv('data/transition_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))
rmse_dff = []
for dff in range(5,80):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)-15
            diff = dff
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-15-diff:k-15])
                l.append(sum(price[k-15:k])/15)
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':1, 
            'bagging_freq':1,
            'feature_fraction':1, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            print(train_x.shape)
            print(test_x.shape)
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        print(rmse)
        print(np.array(rmse).mean())
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    sub['city_code'] = sub[0].map(cc_se)
    sub['district_code'] = sub[1].map(dc_se)
    sub = sub[flow.columns[1:]]
    df = pd.DataFrame()
    for date_valid in flow['date_dt'].drop_duplicates().sort_values().values[-15:]:
        temp = sub.copy()
        temp['date_dt'] = date_valid
        df = pd.concat([df,temp],axis=0,ignore_index=True)
    df = df[flow.columns]

    stat = pd.merge(df,valid,how='left',on=flow.columns.tolist()[:3])

    def getScore(dt):
        sle = 0
        for co in flow.columns[-3:]:
            dt[co+'_x'] = np.log(dt[co+'_x']+1)
            dt[co+'_y'] = np.log(dt[co+'_y']+1)
            sle += ((dt[co+'_x']-dt[co+'_y'])**2).sum()
        sle/=(15*3*98)
        return np.sqrt(sle)
    score = getScore(stat)
    print(score)
    rmse_dff.append(score)

(23520, 7)
(98, 7)
[40.931482677359945, 41.5780863182701, 36.751296554467565, 44.229769699912076, 44.87914338939667]
41.67395572788127
(23520, 7)
(98, 7)
[65.89673048472804, 71.29893194300692, 68.73688028938022, 76.94259315293904, 73.77445986684165]
71.32991914737917
(23520, 7)
(98, 7)
[67.65902335049566, 73.42032347819494, 63.022147564604495, 73.74429258743908, 72.33043961851165]
70.03524531984917
0.12130297590570356
(23422, 8)
(98, 8)
[39.74852447384762, 37.01576340075459, 43.04468212394758, 42.85665656049997, 43.82915356109304]
41.29895602402856
(23422, 8)
(98, 8)
[69.31095128385269, 59.380434498713434, 69.4629662071866, 75.59127820588483, 74.46526671008243]
69.642179381144
(23422, 8)
(98, 8)
[69.52354446978356, 60.098696280920784, 68.13138970154986, 77.77838922935504, 75.65868684904905]
70.23814130613167
0.12067025904384682
(23324, 9)
(98, 9)
[40.32598130197035, 41.22519831131872, 40.88514667405283, 41.33811172216943, 41.228566254513105]
41.00060085280489
(23324, 9)
(98, 9)
[65.717

[65.78016847160436, 65.19487730181115, 68.1898001432566, 66.74694329803438, 64.31243618111898]
66.04484507916509
(21658, 26)
(98, 26)
[66.19360374522101, 66.05732302294692, 67.3203382616551, 64.2076365790021, 69.91007796028454]
66.73779591382193
0.11987699670513911
(21560, 27)
(98, 27)
[37.879277933567685, 35.87162933290226, 39.49913904027221, 41.68179767711605, 41.02484885066426]
39.19133856690449
(21560, 27)
(98, 27)
[64.06809775424642, 62.20158143312402, 64.83569498269516, 72.71299744381365, 66.87545208599099]
66.13876473997405
(21560, 27)
(98, 27)
[62.42048612730904, 62.31879685940732, 69.12350047886594, 65.81384054163901, 69.47027073570884]
65.82937894858603
0.11899359182772269
(21462, 28)
(98, 28)
[44.36683428753165, 36.11861929185786, 35.78539811773008, 46.779396898411605, 34.94766642544529]
39.5995830041953
(21462, 28)
(98, 28)
[68.3411833386636, 61.145717211610894, 60.51817237698842, 77.0984432101462, 60.07430975882901]
65.43556517924762
(21462, 28)
(98, 28)
[75.40573863085817

(19796, 45)
(98, 45)
[60.12716920333858, 57.03399381823349, 55.773580094422925, 54.42198652523987, 59.62494154506336]
57.396334237259644
0.1390208290684561
(19698, 46)
(98, 46)
[38.96365838739824, 33.64868005588535, 33.53921092862062, 32.43618077379122, 36.22988983005937]
34.963523995150965
(19698, 46)
(98, 46)
[51.034673536117566, 55.03919069703061, 56.483771064350755, 56.213038730444104, 60.852590789389204]
55.92465296346645
(19698, 46)
(98, 46)
[58.68424281458564, 61.36667142545155, 57.16225319398652, 59.89404268325794, 61.80162004695847]
59.78176603284802
0.1405273806640613
(19600, 47)
(98, 47)
[35.51380202637068, 33.35472485524613, 28.37706817406397, 37.053587960758435, 37.8427713202785]
34.42839086734354
(19600, 47)
(98, 47)
[54.71340195764966, 56.591665981963324, 53.20592800810377, 59.52224986962892, 56.757362062454376]
56.15812157596001
(19600, 47)
(98, 47)
[57.654489059196386, 57.268094123976816, 52.371183418403355, 66.04012980480123, 59.10323690583303]
58.48742666244216
0.142

(17836, 65)
(98, 65)
[33.431972900710136, 33.41412372008369, 34.947545874106346, 31.936409569873547, 31.534220510412755]
33.05285451503729
(17836, 65)
(98, 65)
[51.662121667923024, 60.78159889246606, 57.380364155661994, 59.67594190856861, 53.690654957730644]
56.63813631647006
(17836, 65)
(98, 65)
[56.17194555286257, 65.07324316428759, 58.92026543780752, 64.13526254110451, 54.04680411007257]
59.66950416122695
0.13272758697627005
(17738, 66)
(98, 66)
[40.747285239407766, 33.76390130418004, 34.2649278159203, 35.295255818671556, 34.990316124416985]
35.81233726051933
(17738, 66)
(98, 66)
[57.62676993582998, 53.6462014305231, 58.440793788148234, 63.6333623837744, 51.65241847277203]
56.99990920220955
(17738, 66)
(98, 66)
[63.77734373436035, 56.73522650855174, 66.0831309537484, 62.62029785185963, 55.88967837050721]
61.02113548380547
0.1300648880600728
(17640, 67)
(98, 67)
[30.67417696452916, 38.31493431630656, 26.857064746177887, 30.478355997124822, 39.58179146809571]
33.181264698446824
(17640

In [2]:
rmse_dff

NameError: name 'rmse_dff' is not defined

In [ ]:
df.to_csv('result/submission_30.csv',index=False,header=None)
按照新旧来调整权重
按照时间序列来做标签
利用arima模型
试试分开建模